##  Data from File 5.2 Bank Accounts

### Step 1. Load required packages

In [1]:
import pandas as pd
import pandasql as ps
from pathlib import Path

#### Step 2 - Get sample from source file

In [2]:
sourceFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/5_Suppliers_Payees_BankAccounts'
workFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/Work/5_Bank_Accounts'
fileName = '5.2. SUPPLIER_BANK_ACCOUNTS_COPPEL.csv'
inputTextFile = open(Path(sourceFolder, fileName), 'r', encoding='latin-1')
if inputTextFile:
    numTextLines = 0
    while True:
        numTextLines += 1
        textLine = inputTextFile.readline()
        print(textLine)
        if numTextLines >10:
            break
    inputTextFile.close()

'SISTEMA_LEGADO';'NUMERO_PROVEEDOR';'FEEDER_IMPORT_BATCH_ID';'TEMP_EXT_PAYEE_ID';'TEMP_EXT_BANK_ACCT_ID';'BANK_NAME';'BRANCH_NAME';'COUNTRY_CODE';'BANK_ACCOUNT_NAME';'BANK_ACCOUNT_NUM';'CURRENCY_CODE';'FOREING_PAYMENT_USE_FLAG';'START_DATE';'END_DATE';'IBAN';'CHECK_DIGITS';'BANK_ACCOUNT_NAME_ALT';'BANK_ACCOUNT_TYPE';'ACCOUNT_SUFFIX';'DESCRIPTION';'AGENCY_LOCATION_CODE';'EXCHANGE_RATE_AGREEMENT_NUM';'EXCHANGE_RATE_AGREEMENT_TYPE';'EXCHANGE_RATE';'SECONDARY_ACCOUNT_REFERENCE';'ATTRIBUTE_CATEGORY';'ATTRIBUTE1';'ATTRIBUTE2';'ATTRIBUTE3';'ATTRIBUTE4';'ATTRIBUTE5';'ATTRIBUTE6';'ATTRIBUTE7';'ATTRIBUTE8';'ATTRIBUTE9';'ATTRIBUTE10';'ATTRIBUTE11';'ATTRIBUTE12';'ATTRIBUTE13';'ATTRIBUTE14';'ATTRIBUTE15'

'OBRAS';'GAVE860528F18';'100';'9';'0009';'HSBC';'PROVEEDORES';'MX';'';'4057367294';'MXN';'N';'';'';'';'';'';'';'';'';'';'';'';'';'';'';'021741040573672946';'';'';'';'';'';'';'';'';'';'';'';'';'';''

'GASTOS';'BIO111108JC6';'100';'12';'00012';'BANORTE';'PROVEEDORES';'MX';'';'0808351359';'MXN';'N';'

### Step 3 - Load bank account data

In [3]:
df = pd.read_csv(Path(sourceFolder,fileName), sep=';', quotechar=chr(39), encoding='latin-1')
df.dtypes

SISTEMA_LEGADO                   object
NUMERO_PROVEEDOR                 object
FEEDER_IMPORT_BATCH_ID            int64
TEMP_EXT_PAYEE_ID                 int64
TEMP_EXT_BANK_ACCT_ID             int64
BANK_NAME                        object
BRANCH_NAME                      object
COUNTRY_CODE                     object
BANK_ACCOUNT_NAME               float64
BANK_ACCOUNT_NUM                float64
CURRENCY_CODE                    object
FOREING_PAYMENT_USE_FLAG         object
START_DATE                      float64
END_DATE                        float64
IBAN                            float64
CHECK_DIGITS                    float64
BANK_ACCOUNT_NAME_ALT           float64
BANK_ACCOUNT_TYPE               float64
ACCOUNT_SUFFIX                  float64
DESCRIPTION                     float64
AGENCY_LOCATION_CODE            float64
EXCHANGE_RATE_AGREEMENT_NUM     float64
EXCHANGE_RATE_AGREEMENT_TYPE    float64
EXCHANGE_RATE                   float64
SECONDARY_ACCOUNT_REFERENCE     float64


### Step 4 - Validate Bank Account & Tax ID

In [30]:
modDf = df
modDf['CLABE_VALIDATION'] = False
modDf['BANK_ACCOUNT_NUM'] = modDf['BANK_ACCOUNT_NUM'].astype('|S') 
modDf['ATTRIBUTE1'] = modDf['ATTRIBUTE1'].astype('|S') 
for index, row in modDf.iterrows():
    if len(str(row['ATTRIBUTE1']).replace('.0',''))==18:
        modDf.loc[index, 'CLABE_VALIDATION'] = True
modDf = modDf[modDf['NUMERO_PROVEEDOR'].notna()]
print('the orignal dataset contains ', len(df.index), ' records')
print('the modified dataset contains ', len(modDf.index), ' records.')
print(modDf.head(10))
print(modDf.dtypes)

the orignal dataset contains  171966  records
the modified dataset contains  171966  records.
  SISTEMA_LEGADO NUMERO_PROVEEDOR  FEEDER_IMPORT_BATCH_ID  TEMP_EXT_PAYEE_ID  \
0          OBRAS    GAVE860528F18                     100                  9   
1         GASTOS     BIO111108JC6                     100                 12   
2          OBRAS     BIO111108JC6                     100                 13   
3         GASTOS     DSE090511EW5                     100                 18   
4          OBRAS     DSE090511EW5                     100                 19   
5          OBRAS     TVC8607234U2                     100                 23   
6         GASTOS     TVC8607234U2                     100                 24   
7         GASTOS     ABS110113195                     100                 27   
8          OBRAS     ABS110113195                     100                 28   
9          OBRAS     DUO7511286H3                     100                 37   

   TEMP_EXT_BANK_ACCT_ID 

### Step 5 - Creates a subset with elements whose taxt id and bank account are valid

In [31]:
subSetDf = modDf[modDf['CLABE_VALIDATION']==True]
subSetDf['BANK_ACCOUNT_NUM'] = subSetDf['BANK_ACCOUNT_NUM'].replace('.0','')
subSetDf['ATTRIBUTE1'] = subSetDf['ATTRIBUTE1'].replace('.0','')
print(len(modDf.index) ,' records in original dataset')
print(len(subSetDf.index), ' records with a valid bank account (18 digits)')
print(subSetDf.head(10))

171966  records in original dataset
4  records with a valid bank account (18 digits)
       SISTEMA_LEGADO NUMERO_PROVEEDOR  FEEDER_IMPORT_BATCH_ID  \
149936          OBRAS     SAP8203102M4                     100   
153487         GASTOS     SAP8203102M4                     100   
153497     CONSTRUNET            29336                     100   
153498            SIM            29336                     100   

        TEMP_EXT_PAYEE_ID  TEMP_EXT_BANK_ACCT_ID      BANK_NAME  BRANCH_NAME  \
149936             206143                   7169  BBVA BANCOMER  PROVEEDORES   
153487             211689                  12610  BBVA BANCOMER  PROVEEDORES   
153497             211699                  12620  BBVA BANCOMER  PROVEEDORES   
153498             211700                  12621  BBVA BANCOMER  PROVEEDORES   

       COUNTRY_CODE  BANK_ACCOUNT_NAME BANK_ACCOUNT_NUM  ... ATTRIBUTE7  \
149936           MX                NaN       b'66608.0'  ...        NaN   
153487           MX              

### Step 6 - Performs analysis for duplicate records based on taxt id or supplier id

In [32]:
sql_query = '''
SELECT NUMERO_PROVEEDOR,COUNT(*) AS numObs FROM subSetDf
GROUP BY NUMERO_PROVEEDOR
ORDER BY numObs DESC
'''
dupsAnalysis = ps.sqldf(sql_query)
print(len(dupsAnalysis.index), ' taxt ids in daset')
uniqueTaxdIds = dupsAnalysis[dupsAnalysis['numObs']==1]
print(len(uniqueTaxdIds.index), ' unique taxt ids')
duplicateTaxdIds = dupsAnalysis[dupsAnalysis['numObs']>1]
print(len(duplicateTaxdIds.index), ' duplicate taxt ids')
#
sql_query = '''
SELECT SISTEMA_LEGADO,COUNT(*) AS numObs FROM 
(SELECT SISTEMA_LEGADO,NUMERO_PROVEEDOR FROM subSetDf)
GROUP BY SISTEMA_LEGADO
ORDER BY numObs DESC
'''
distPerLegacy = ps.sqldf(sql_query)
for index, row in distPerLegacy.iterrows():
    print(row['SISTEMA_LEGADO'], ',', row['numObs'])

2  taxt ids in daset
0  unique taxt ids
2  duplicate taxt ids
CONSTRUNET , 1
GASTOS , 1
OBRAS , 1
SIM , 1


In [33]:
print('unique taxt ids: ')
print(uniqueTaxdIds.head(10))
print('duplicate taxt ids: ')
print(duplicateTaxdIds.head(100))

unique taxt ids: 
Empty DataFrame
Columns: [NUMERO_PROVEEDOR, numObs]
Index: []
duplicate taxt ids: 
  NUMERO_PROVEEDOR  numObs
0            29336       2
1     SAP8203102M4       2


In [35]:
sql_query = '''
SELECT numObs,COUNT(*) AS NumCases FROM duplicateTaxdIds
GROUP BY numObs
ORDER BY numObs DESC
'''
result = ps.sqldf(sql_query)
for index, row in result.iterrows():
    print(row['numObs'], ' duplicates, ', row['NumCases'], ' cases')

2  duplicates,  2  cases


### Step 7 - Perfoms analysis of distribution of accounts per supplier

In [36]:
sql_query = '''
SELECT DISTINCT NUMERO_PROVEEDOR,ATTRIBUTE1 FROM subSetDf
ORDER BY NUMERO_PROVEEDOR
'''
dataInScopeDf = ps.sqldf(sql_query)
print(len(dataInScopeDf.index), ' records in scope')
dataInScopeDf.to_csv(Path(workFolder, 'dataInScopeDfs.csv'), encoding='utf-8')
sql_query = '''
SELECT NUMERO_PROVEEDOR,COUNT(*) AS numObs FROM dataInScopeDf
GROUP BY NUMERO_PROVEEDOR
ORDER BY numObs DESC
'''
accountsPerSupplierDf = ps.sqldf(sql_query)
print(len(accountsPerSupplierDf.index), ' suppliers in dataset')
sql_query = '''
SELECT numObs,COUNT(*) AS numCases FROM accountsPerSupplierDf
GROUP BY numObs
ORDER BY numObs DESC
'''
result = ps.sqldf(sql_query)
for index, row in result.iterrows():
    print(row['numObs'], ', ', row['numCases'])
#
sql_query = '''
SELECT DISTINCT SISTEMA_LEGADO FROM subSetDf
''' 
legacySystemsInScopeDf = ps.sqldf(sql_query)
print(len(legacySystemsInScopeDf.index), ' legacy sistems in scope')
legacySystemsInScopeDf.to_csv(Path(workFolder, 'legacySystemsInScopeDfs.csv'), encoding='utf-8')

2  records in scope
2  suppliers in dataset
1 ,  2
4  legacy sistems in scope


### Step 8 - Create new reporting structure from subset

In [10]:
newReportDf = subSetDf[['SISTEMA_LEGADO','NUMERO_PROVEEDOR','FEEDER_IMPORT_BATCH_ID','TEMP_EXT_PAYEE_ID','TEMP_EXT_BANK_ACCT_ID',
'BANK_NAME','BRANCH_NAME','COUNTRY_CODE','BANK_ACCOUNT_NAME','BANK_ACCOUNT_NUM','CURRENCY_CODE','FOREING_PAYMENT_USE_FLAG',
'START_DATE','END_DATE','IBAN','CHECK_DIGITS','BANK_ACCOUNT_NAME_ALT','BANK_ACCOUNT_TYPE','ACCOUNT_SUFFIX','DESCRIPTION',
'AGENCY_LOCATION_CODE','EXCHANGE_RATE_AGREEMENT_NUM','EXCHANGE_RATE_AGREEMENT_TYPE','EXCHANGE_RATE','SECONDARY_ACCOUNT_REFERENCE',
'ATTRIBUTE_CATEGORY','ATTRIBUTE1','ATTRIBUTE2','ATTRIBUTE3','ATTRIBUTE4','ATTRIBUTE5','ATTRIBUTE6','ATTRIBUTE7','ATTRIBUTE8',
'ATTRIBUTE9','ATTRIBUTE10','ATTRIBUTE11','ATTRIBUTE12','ATTRIBUTE13','ATTRIBUTE14','ATTRIBUTE15']]
print(len(newReportDf.index), ' in new report structure')

164836  in new report structure


### Step 9 - Create new textfile

In [29]:
print(newReportDf.dtypes)

SISTEMA_LEGADO                  object
NUMERO_PROVEEDOR                object
FEEDER_IMPORT_BATCH_ID           int64
TEMP_EXT_PAYEE_ID                int64
TEMP_EXT_BANK_ACCT_ID            int64
BANK_NAME                       object
BRANCH_NAME                     object
COUNTRY_CODE                    object
BANK_ACCOUNT_NAME               object
BANK_ACCOUNT_NUM                object
CURRENCY_CODE                   object
FOREING_PAYMENT_USE_FLAG        object
START_DATE                      object
END_DATE                        object
IBAN                            object
CHECK_DIGITS                    object
BANK_ACCOUNT_NAME_ALT           object
BANK_ACCOUNT_TYPE               object
ACCOUNT_SUFFIX                  object
DESCRIPTION                     object
AGENCY_LOCATION_CODE            object
EXCHANGE_RATE_AGREEMENT_NUM     object
EXCHANGE_RATE_AGREEMENT_TYPE    object
EXCHANGE_RATE                   object
SECONDARY_ACCOUNT_REFERENCE     object
ATTRIBUTE_CATEGORY       

In [28]:
#newReportDf.fillna('', inplace=True)
#with open(Path(workFolder,'5.2 SUPPLIER_BANK_ACCOUNTS_COPPEL(NEW).csv'), 'w', encoding='utf-8') as textFile:
#    headers = ['SISTEMA_LEGADO','NUMERO_PROVEEDOR','FEEDER_IMPORT_BATCH_ID','TEMP_EXT_PAYEE_ID','TEMP_EXT_BANK_ACCT_ID',
#               'BANK_NAME','BRANCH_NAME','COUNTRY_CODE','BANK_ACCOUNT_NAME','BANK_ACCOUNT_NUM','CURRENCY_CODE','FOREING_PAYMENT_USE_FLAG',
#               'START_DATE','END_DATE','IBAN','CHECK_DIGITS','BANK_ACCOUNT_NAME_ALT','BANK_ACCOUNT_TYPE','ACCOUNT_SUFFIX','DESCRIPTION',
#               'AGENCY_LOCATION_CODE','EXCHANGE_RATE_AGREEMENT_NUM','EXCHANGE_RATE_AGREEMENT_TYPE','EXCHANGE_RATE','SECONDARY_ACCOUNT_REFERENCE',
#               'ATTRIBUTE_CATEGORY','ATTRIBUTE1','ATTRIBUTE2','ATTRIBUTE3','ATTRIBUTE4','ATTRIBUTE5','ATTRIBUTE6','ATTRIBUTE7','ATTRIBUTE8',
#               'ATTRIBUTE9','ATTRIBUTE10','ATTRIBUTE11','ATTRIBUTE12','ATTRIBUTE13','ATTRIBUTE14','ATTRIBUTE15']
#    textLine = chr(39).join(headers) + chr(10)
#    textFile.write(textLine)
#    for index, row in newReportDf.iterrows():
#        tmpVector = row.to_list()
#        textLine = chr(39).join(tmpVector) + chr(10)
#        textFile.write(textLine)
#        if index > 10: break
#    textFile.close()
for index, row in newReportDf.iterrows():
    tmpVector = row.to_list()
    print(tmpVector)
    tmpVector[9] = str(tmpVector[9]).replace('.0','')
    textLine = chr(39).join([str(tmpColumn) for tmpColumn in tmpVector])
    if index > 10: break


['OBRAS', 'GAVE860528F18', 100, 9, 9, 'HSBC', 'PROVEEDORES', 'MX', '', '4057367294.0', 'MXN', 'N', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '021741040573672946', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['GASTOS', 'BIO111108JC6', 100, 12, 12, 'BANORTE', 'PROVEEDORES', 'MX', '', '808351359.0', 'MXN', 'N', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '072180008083513590', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['OBRAS', 'BIO111108JC6', 100, 13, 13, 'BANORTE', 'PROVEEDORES', 'MX', '', '808351359.0', 'MXN', 'N', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '072180008083513590', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['GASTOS', 'DSE090511EW5', 100, 18, 18, 'AFIRME', 'PROVEEDORES', 'MX', '', '108119934.0', 'MXN', 'N', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '062580001081199345', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['OBRAS', 'DSE090511EW5', 100, 19, 19, 'AFIRME', 'PROVEEDO